In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import torch
torch.cuda.is_available()

/insomnia001/home/aa5506/miniconda3/envs/spec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [2]:
import sys
print(sys.executable)

/insomnia001/home/aa5506/miniconda3/envs/spec/bin/python


In [2]:
prompt = "The meaning of life is"

target_cpt = "google/gemma-2b"
assistant_cpt = "double7/vicuna-68m"

In [3]:
target_tokenizer = AutoTokenizer.from_pretrained(target_cpt)
assistant_tokenizer = AutoTokenizer.from_pretrained(assistant_cpt)

target_model = AutoModelForCausalLM.from_pretrained(target_cpt)
assistant_model = AutoModelForCausalLM.from_pretrained(assistant_cpt)

Loading checkpoint shards: 100%|██████████| 2/2 [00:39<00:00, 19.77s/it]


In [4]:
def compare(prompt_list):
    
    assisted_times = []
    regular_times = []
    
    for prompt in prompt_list:
        inputs = target_tokenizer(prompt, return_tensors='pt')
    
        s = time.time()
        
        outputs = target_model.generate(**inputs, assistant_model=assistant_model, tokenizer=target_tokenizer, assistant_tokenizer=assistant_tokenizer)
        target_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        assisted_times.append(time.time()-s)
        s = time.time()
        
        outputs = target_model.generate(**inputs, tokenizer=target_tokenizer)
        target_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        regular_times.append(time.time()-s)
        
    return sum(assisted_times)/sum(regular_times)

In [5]:
prompt_list = [
    'What is the capital of France?',
    'Who wrote \'1984\'?',
    'The quick brown fox jumps over the lazy...',
    'Write a Python function to compute the Fibonacci sequence.',
    'Define Newton\'s second law of motion.',
    'List three primary colors.',
    'Convert 100°F to Celsius.',
    'What are the three branches of the U.S. government?',
    'How do you define a class in Python?',
    'What\'s 8 times 7?'
]

In [6]:
compare(prompt_list)

1.35325377496163